In [115]:
import os
from os.path import isfile, join, dirname, abspath
import sys
import re
import argparse
import shutil
import subprocess
import tempfile
import pandas as pd

import shapefile
from json import dumps

In [116]:
reader = shapefile.Reader("../data/gis/stations3d.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []

for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    # filter by centerline
    
    if 'ENT' in atr['_NAME']:      
        print(atr)
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr))
    # write the GeoJSON file

#geojson = open("../data/gis/points_fixes.js", "w")
#geojson.write("var points_fixes = \n")
#geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
#geojson.close()


{'_ID': 1, '_UID': 1, '_NAME': 'ENT_101001', '_SURVEY': '101.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Abri 2, occupation humaine'}
{'_ID': 2, '_UID': 2, '_NAME': 'ENT_102001', '_SURVEY': '102.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Porche'}
{'_ID': 3, '_UID': 3, '_NAME': 'ENT_102002', '_SURVEY': '102.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Punta Espolon'}
{'_ID': 4, '_UID': 4, '_NAME': 'ENT_104001', '_SURVEY': '104.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Abri 1, occupation humaine'}
{'_ID': 5, '_UID': 5, '_NAME': 'ENT_104002', '_SURVEY': '104.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Cueva rien'}
{'_ID': 6, '_UID': 6, '_NAME': 'ENT_105001', '_SURVEY': '105.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0,

In [119]:

THERION_STATUS_LIST = [(feature['properties']['_NAME'],feature['properties']['_COMMENT'],feature['geometry']['coordinates'][0],feature['geometry']['coordinates'][1]) for feature in buffer]

In [120]:
THERION_STATUS_LIST

[('ENT_101001', 'Abri 2, occupation humaine', 476162.0, 4461336.0),
 ('ENT_102001', 'Porche', 476260.0, 4449342.0),
 ('ENT_102002', 'Punta Espolon', 474766.0, 4451367.0),
 ('ENT_104001', 'Abri 1, occupation humaine', 475478.0, 4450597.0),
 ('ENT_104002', 'Cueva rien', 475729.0, 4450525.0),
 ('ENT_105001', 'Cueva Rapido1', 482677.0, 4454593.0),
 ('ENT_105002', 'Cueva Rapido 2', 482677.0, 4454593.0),
 ('ENT_106001', 'Cueva Pirates/ Hacienda Salinas', 490220.0, 4452363.0),
 ('ENT_106002', 'Cueva No Time', 490289.0, 4451286.0),
 ('ENT_106003', 'Résurgence NoTime', 490289.0, 4451286.0),
 ('ENT_106004', 'Cueva Nutria', 490541.0, 4453870.0),
 ('ENT_106005', 'Porche', 490289.0, 4451286.0),
 ('ENT_106006', 'Grotte', 490604.0, 4450548.0),
 ('ENT_106008', 'Caracara', 490080.0, 4452381.0),
 ('ENT_106009',
  'Plateau Egg 1, non descendu à cause de blocs instables',
  488815.0,
  4453011.0),
 ('ENT_106010',
  'Plateau Egg 2, environ 40 m avec un méandre rétrécissant',
  488956.0,
  4452962.0),
 ('EN

In [121]:
## read the cavity synthesis
synthese_cavites = pd.read_csv("../data/BROUILLON_cadastre/UP_MDD_DDA_Temp_synthese_Cavites.csv")

In [133]:
# index

CADASTRE_NUMBER = []
for element in synthese_cavites[['X (UTM 18 Sud)','Y (UTM 18 Sud)','NomCadastre']].iterrows():
    LINE = []
    LINE.append(element[1][2])
    for fixed_point in THERION_STATUS_LIST:
        try:
            if int(fixed_point[2]) == int(element[1][0]):
                if int(fixed_point[3]) == int(element[1][1]):
                    print(fixed_point[0],element[1][2])
                    LINE.append(fixed_point[0])

        except ValueError:
            #print("coordinates are likely NaN")
            pass
    CADASTRE_NUMBER.append(LINE)

ENT_213001 LeMasqueEtLaPlume
ENT_211001 AbrigoDelViento
ENT_211002 SumideroPiquante
ENT_211003 GouffreAguilucho
ENT_211004 ExsurgencePicaflor
ENT_211006 SimaDelCol
ENT_211007 MD-BDGS01
ENT_211008 MD-BDGS02
ENT_211009 MD-BDGS03
ENT_211010 MD-BDGS04
ENT_211011 MD-BDG05
ENT_209001 PleinCintre
ENT_209002 EmergenceBarillet
ENT_209003 GouffreDeLaGachette
ENT_209004 Perte de l'Acharné
ENT_209005 Perte n° 18
ENT_209006 ResurgenceDeLaPlage
ENT_209007 GrotteEvier
ENT_209009 CuevaGringa
ENT_208001 Cueva10H10
ENT_205001 AbriCotier
ENT_208002 Résurgence 9 plage de la Baleine
ENT_208003 Grotte20PlageDeLaBaleine
ENT_205002 AbriDuCerfeuil
ENT_206001 CuevaAyayema
ENT_206002 GrotteDuDyke
ENT_206044 ResurgenceDuChampignon
ENT_205003 CuevaLaBas
ENT_205004 PerteDesDents
ENT_205005 SumideroJuanito
ENT_205006 CuevaArdilla
ENT_205007 PozoEnBosque
ENT_205033 PozoEnBosque
ENT_205008 Pozo 2 en Caliza
ENT_205032 Pozo 2 en Caliza
ENT_205009 Pozo 1 en Caliza
ENT_205034 Pozo 1 en Caliza
ENT_205010 MD-SOU01
ENT_20501

ENT_209040 PozoAlRato
ENT_209041 PozoDeLosCinquetenos
ENT_209058 PerteDesSabliers
ENT_207001 SystemeCondors
ENT_207001 SystemeCondors
ENT_207001 GouffrePetitCreux


In [135]:
CADNUM = []
for element in CADASTRE_NUMBER:
    try:
        CADNUM.append(element[1])
    except IndexError:
        print('no cad num attribué')
        CADNUM.append('NAN')
        print(element[0])
        pass

no cad num attribué
PorcheDeLImpatience
no cad num attribué
PequinaSud
no cad num attribué
Perte sans nom
no cad num attribué
BobLEponge
no cad num attribué
Grotte du bout du monde
no cad num attribué
Résurgence 3 plage de la Baleine
no cad num attribué
Résurgence 5 plage de la Baleine
no cad num attribué
CuevaPirates
no cad num attribué
CuevaDeLasLapas
no cad num attribué
GoldenIgue
no cad num attribué
PerteNotoCanada
no cad num attribué
GrotteCacahuete
no cad num attribué
PorcheDeLaCrevette
no cad num attribué
PerteJeanneDArc
no cad num attribué
cueva103
no cad num attribué
Résurgence
no cad num attribué
Résurgence
no cad num attribué
Résurgence
no cad num attribué
GouffreInstantané
no cad num attribué
GouffreDuCanyon
no cad num attribué
LosPiscoleros
no cad num attribué
Traversé du roberto
no cad num attribué
trou noir
no cad num attribué
SumideroArdilla
no cad num attribué
GrotteAudeline
no cad num attribué
Perte1
no cad num attribué
TrouQuiPayePasDeMine
no cad num attribué
Resurge

In [136]:
synthese_cavites['CAD_NUM'] = CADNUM

101001
    Unnamed: 0    UP Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
54          54  2008        Abri        476162.0       4461336.0    2   

    Dev. Topo Dev.estimé Prof.    Topo.  ... Unnamed: 28 Unnamed: 29  \
54        NaN        NaN   NaN  croquis  ...         NaN         NaN   

   Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
54         NaN         NaN          476.162          4461.336     476162.0   

    Unnamed: 35  NomCadastre     CAD_NUM  
54    4461336.0        Abri2  ENT_101001  

[1 rows x 39 columns]
102001
    Unnamed: 0    UP Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
53          53  2008      Porche        476260.0       4449342.0   30   

    Dev. Topo Dev.estimé Prof.    Topo.  ... Unnamed: 28 Unnamed: 29  \
53        NaN         30   NaN  croquis  ...         NaN         NaN   

   Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
53         NaN         NaN           476.26          4449.

[1 rows x 39 columns]
107003
     Unnamed: 0    UP                   Nom_complet  X (UTM 18 Sud)  \
340         340  2017  Deux grottes (double entree)        478906.0   

     Y (UTM 18 Sud) Alt.  Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28  \
340       4446437.0    ?        NaN          5     ?   non  ...         NaN   

    Unnamed: 29 Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  \
340         NaN         NaN         NaN          478.906          4446.437   

     Unnamed: 34  Unnamed: 35  NomCadastre     CAD_NUM  
340     478906.0    4446437.0  DeuxGrottes  ENT_107003  

[1 rows x 39 columns]
107004
     Unnamed: 0    UP           Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud)  \
341         341  2017  Chattiere qui buffle        482761.0       4445427.0   

    Alt.  Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
341    ?        NaN        100     ?   non  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Su

     Unnamed: 0    UP      Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
317         317  2017  Ojo de la Madre        484150.0       4446766.0  406   

     Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
317      190.0        190    49   oui  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
317         NaN         NaN           484.15          4446.766     484150.0   

     Unnamed: 35   NomCadastre     CAD_NUM  
317    4446766.0  OjoDeLaMadre  ENT_108001  

[1 rows x 39 columns]
108002
     Unnamed: 0    UP          Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud)  \
318         318  2017  Ombligo de la Madre        484295.0       4447150.0   

    Alt.  Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
318  365      283.0        283   146   oui  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
318         NaN         NaN       

     Unnamed: 0    UP          Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud)  \
396         396  2019  Gouffre Ma première        485587.0       4445498.0   

    Alt.  Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
396  319        NaN         25    25   NaN  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
396         NaN         NaN          485.587          4445.498     485587.0   

     Unnamed: 35        NomCadastre     CAD_NUM  
396    4445498.0  GouffreMaPremiere  ENT_109010  

[1 rows x 39 columns]
109011
     Unnamed: 0    UP                  Nom_complet  X (UTM 18 Sud)  \
397         397  2019  Gouffre La Grande Traversée        485584.0   

     Y (UTM 18 Sud) Alt.  Dev. Topo Dev.estimé    Prof. Topo.  ...  \
397       4445478.0  319        NaN         86  -4, +11   NaN  ...   

    Unnamed: 28 Unnamed: 29 Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  \
397         NaN         NaN         NaN         N

     Unnamed: 0    UP         Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud)  \
112         112  2010  Cueva du Téléphone        473512.0       4437140.0   

    Alt.  Dev. Topo Dev.estimé Prof.    Topo.  ... Unnamed: 28 Unnamed: 29  \
112  340        0.0         38   -26  croquis  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
112         NaN         NaN          473.512           4437.14     473512.0   

     Unnamed: 35       NomCadastre     CAD_NUM  
112    4437140.0  CuevaDuTelephone  ENT_201009  

[1 rows x 39 columns]
201010
     Unnamed: 0    UP Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
113         113  2010     Cueva 8        473749.0       4437328.0  309   

     Dev. Topo Dev.estimé Prof.    Topo.  ... Unnamed: 28 Unnamed: 29  \
113        0.0         30   -26  croquis  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
113         NaN         Na

...

[2 rows x 39 columns]
211045
     Unnamed: 0    UP          Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud)  \
206         206  2010  El Culo de la Madre        468699.0       4422722.0   

    Alt.  Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
206  355        NaN        182   -78  topo  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
206         NaN         NaN          468.699          4422.722     468699.0   

     Unnamed: 35      NomCadastre     CAD_NUM  
206    4422722.0  ElCuloDeLaMadre  ENT_211045  

[1 rows x 39 columns]
211046
     Unnamed: 0    UP Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
207         207  2010   La Grotte        468496.0       4422512.0  331   

     Dev. Topo Dev.estimé Prof.    Topo.  ... Unnamed: 28 Unnamed: 29  \
207        0.0          9    -9  croquis  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
207

[1 rows x 39 columns]
303009
     Unnamed: 0    UP       Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
268         268  2014  Cueva du Plateau        493877.0       4308702.0  NaN   

     Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
268        NaN         20   -20   NaN  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
268         NaN         NaN          493.877          4308.702     493877.0   

     Unnamed: 35     NomCadastre     CAD_NUM  
268    4308702.0  CuevaDuPlateau  ENT_303009  

[1 rows x 39 columns]
303010
     Unnamed: 0    UP   Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
269         269  2014  Igue du Dike        493877.0       4308689.0  NaN   

     Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
269        NaN         20   -20   NaN  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
269         N

[1 rows x 39 columns]
304015
     Unnamed: 0    UP     Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
266         266  2014  Perte du col 1        493740.0       4308750.0  NaN   

     Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
266        NaN        -30   -30   NaN  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
266         NaN         NaN           493.74           4308.75     493740.0   

     Unnamed: 35  NomCadastre     CAD_NUM  
266    4308750.0  PerteDuCol1  ENT_304015  

[1 rows x 39 columns]
304016
     Unnamed: 0    UP     Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
267         267  2014  Perte du col 2        493735.0       4308575.0  NaN   

     Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
267        NaN          0     0   NaN  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  Unnamed: 34  \
267         NaN    

[1 rows x 39 columns]
304037
     Unnamed: 0    UP                    Nom_complet  X (UTM 18 Sud)  \
293         293  2014  Gouffre près du camp du Futur        494155.0   

     Y (UTM 18 Sud) Alt.  Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28  \
293       4303338.0  407        NaN         20   -20   NaN  ...         NaN   

    Unnamed: 29 Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  \
293         NaN         NaN         NaN          494.155          4303.338   

     Unnamed: 34  Unnamed: 35                    NomCadastre     CAD_NUM  
293     494155.0    4303338.0  Gouffre près du camp du Futur  ENT_304037  

[1 rows x 39 columns]
304038
     Unnamed: 0    UP Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
294         294  2014      Arc 14        492564.0       4307349.0  347   

     Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
294        NaN         60   -60   NaN  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Su

[1 rows x 39 columns]
401007
     Unnamed: 0    UP              Nom_complet  X (UTM 18 Sud)  \
420         420  2019  Traversée de la Cascade        576183.0   

     Y (UTM 18 Sud) Alt.  Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28  \
420       4600809.0  NaN        NaN         12     3   NaN  ...         NaN   

    Unnamed: 29 Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 Sud).1  \
420         NaN         NaN         NaN          576.183          4600.809   

     Unnamed: 34  Unnamed: 35           NomCadastre     CAD_NUM  
420     576183.0    4600809.0  TraverseeDeLaCascade  ENT_401007  

[1 rows x 39 columns]
401008
     Unnamed: 0    UP        Nom_complet  X (UTM 18 Sud)  Y (UTM 18 Sud) Alt.  \
421         421  2019  L'abri sous glace        576064.0       4600782.0  NaN   

     Dev. Topo Dev.estimé Prof. Topo.  ... Unnamed: 28 Unnamed: 29  \
421        NaN         10     2   NaN  ...         NaN         NaN   

    Unnamed: 30 Unnamed: 31 X (UTM 18 Sud).1  Y (UTM 18 

In [151]:
synthese_cavites

,Unnamed: 0,UP,Nom_complet,X (UTM 18 Sud),Y (UTM 18 Sud),Alt.,Dev. Topo,Dev.estimé,Prof.,Topo.,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,X (UTM 18 Sud).1,Y (UTM 18 Sud).1,Unnamed: 34,Unnamed: 35,NomCadastre,CAD_NUM
0,0,2008,Perte du Masque et de la Plume,473667.0,4416955.0,90,1156.0,NaN,-66,oui,...,NaN,NaN,NaN,NaN,473.667,4416.955,473667.0,4416955.0,LeMasqueEtLaPlume,ENT_213001
1,1,2008,El Abrito del Viento,469367.0,4421504.0,407,430.0,20,-197,oui,...,NaN,NaN,NaN,NaN,469.367,4421.504,469367.0,4421504.0,AbrigoDelViento,ENT_211001
2,2,2008,Sumidero Piquante,469510.0,4421914.0,191,450.0,NaN,-147,oui,...,NaN,NaN,NaN,NaN,469.510,4421.914,469510.0,4421914.0,SumideroPiquante,ENT_211002
3,3,2008,Gouffre d'Aguilucho,470302.0,4421526.0,73,302.0,NaN,-97,oui,...,NaN,NaN,NaN,NaN,470.302,4421.526,470302.0,4421526.0,GouffreAguilucho,ENT_211003
4,4,2008,Exurgence du Picaflor,470298.0,4421336.0,10,40.0,NaN,5,oui,...,NaN,NaN,NaN,NaN,470.298,4421.336,470298.0,4421336.0,ExsurgencePicaflor,ENT_211004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,443,2006,Perte des Sabliers,473093.0,4426314.0,452,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PerteDesSabliers,ENT_209058
445,444,2006,Resurgence des Lobos,479668.0,4427696.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResurgenceLobos,NAN
446,445,2006,Systeme des Condors,479379.0,4428572.0,141,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SystemeCondors,ENT_207001
447,446,2006,Systeme des Condors,479379.0,4428572.0,141,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SystemeCondors,ENT_207001


In [138]:
print(synthese_cavites.columns)

Index(['Unnamed: 0', 'UP', 'Nom_complet', 'X (UTM 18 Sud)', 'Y (UTM 18 Sud)',
       'Alt.', 'Dev. Topo', 'Dev.estimé', 'Prof.', 'Topo.', 'Termine?',
       'Secteur', 'Commentaire', 'Explorateurs', 'ARCHEO', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'X (UTM 18 Sud).1', 'Y (UTM 18 Sud).1', 'Unnamed: 34',
       'Unnamed: 35', 'NomCadastre', 'CAD_NUM'],
      dtype='object')


In [153]:


columns = ['CAD_NUM','NomCadastre','Nom_complet','X (UTM 18 Sud)', 'Y (UTM 18 Sud)','Alt.','Dev. Topo','Termine?', 'Dev.estimé',"Prof.",'UP','Commentaire','Explorateurs']
NOUVELLE_SYNTHESE = synthese_cavites[columns]

In [154]:
NOUVELLE_SYNTHESE.sort_values(by='CAD_NUM').to_csv('../data/BROUILLON_cadastre/UP_MDD_DDA_Temp_synthese_Cavites_NumCad.csv')

In [170]:
NOUVELLE_SYNTHESE.sort_values(by='CAD_NUM')

,CAD_NUM,NomCadastre,Nom_complet,X (UTM 18 Sud),Y (UTM 18 Sud),Alt.,Dev. Topo,Termine?,Dev.estimé,Prof.,UP,Commentaire,Explorateurs
54,ENT_101001,Abri2,Abri,476162.0,4461336.0,2,NaN,oui,NaN,NaN,2008,Occupation humaine,NaN
53,ENT_102001,Porche,Porche,476260.0,4449342.0,30,NaN,oui,30,NaN,2008,3 entrées,NaN
364,ENT_102002,PuntaEspolon,Punta espolon,474766.0,4451367.0,ca 50 m,NaN,NON,200,??,2017,"Exploration Anthony, Yannick, Gelo et Franck, ...",NaN
52,ENT_104001,Abri1,Abri,475478.0,4450597.0,5,NaN,oui,NaN,NaN,2008,Occupation humaine,NaN
55,ENT_104002,CuevaRien,Cueva Rien,475729.0,4450525.0,NaN,NaN,oui,NaN,NaN,2008,3 grottes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NAN,TroisEntreesPlusUne,Grotte des trois entrées plus une,NaN,NaN,?,2434.0,non,2434,48--54,2017,"Cavité énorme, exploration à continuer",NaN
174,NAN,GouffreDuCanyon,Gouffre du Canyon,NaN,NaN,NaN,NaN,non,75,-29,2010,Suite de petits ressauts dans méandre. Arrêt s...,"Arnauld, Denis, Olivier T"
6,NAN,PequinaSud,Pequina Sud,NaN,NaN,NaN,NaN,oui,20,-17,2008,NaN,NaN
21,NAN,BobLEponge,Bob l'Eponge,473372.0,4427993.0,200,NaN,oui,60,-40,2008,"petite perte, arrêt sur siphon",NaN


In [174]:
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    # filter by centerline
    
    if 'ENT' in atr['_NAME']:      
        
        try: 
            vals = (NOUVELLE_SYNTHESE[NOUVELLE_SYNTHESE['CAD_NUM'] == atr['_NAME']].values[0])
            atr['_CAD_NUM'] = vals[0].strip('ENT_')
            atr['_CAVENAME'] = vals[1] 
            atr['_LENGTH'] = vals[6]
            atr['_DEPTH'] = vals[9]
        except IndexError:
            atr['_CAD_NUM'] = atr['_NAME'].strip('ENT_')
            atr['_CAVENAME'] = 'not known'
            atr['_LENGTH'] = "not known"
            atr['_DEPTH'] = "not known"
            pass

        print(atr)
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr))

{'_ID': 1, '_UID': 1, '_NAME': 'ENT_101001', '_SURVEY': '101.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Abri 2, occupation humaine', '_CAD_NUM': '101001', '_CAVENAME': 'Abri2', '_LENGTH': nan, '_DEPTH': nan}
{'_ID': 2, '_UID': 2, '_NAME': 'ENT_102001', '_SURVEY': '102.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Porche', '_CAD_NUM': '102001', '_CAVENAME': 'Porche', '_LENGTH': nan, '_DEPTH': nan}
{'_ID': 3, '_UID': 3, '_NAME': 'ENT_102002', '_SURVEY': '102.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Punta Espolon', '_CAD_NUM': '102002', '_CAVENAME': 'PuntaEspolon', '_LENGTH': nan, '_DEPTH': '??'}
{'_ID': 4, '_UID': 4, '_NAME': 'ENT_104001', '_SURVEY': '104.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Abri 1, occupation humaine', '_CAD_NUM': '104001', '_CAVENAME': 'Abri1', '_LENGTH': nan, '_DEPTH': n

{'_ID': 2448, '_UID': 2448, '_NAME': 'ENT_205006', '_SURVEY': '205.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Cueva Ardilla', '_CAD_NUM': '205006', '_CAVENAME': 'CuevaArdilla', '_LENGTH': nan, '_DEPTH': '-70'}
{'_ID': 2449, '_UID': 2449, '_NAME': 'ENT_205007', '_SURVEY': '205.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Pozo en Bosque', '_CAD_NUM': '205007', '_CAVENAME': 'PozoEnBosque', '_LENGTH': nan, '_DEPTH': '-35'}
{'_ID': 2450, '_UID': 2450, '_NAME': 'ENT_205008', '_SURVEY': '205.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Pozo 2 en Caliza', '_CAD_NUM': '205008', '_CAVENAME': 'Pozo 2 en Caliza', '_LENGTH': nan, '_DEPTH': '-70'}
{'_ID': 2451, '_UID': 2451, '_NAME': 'ENT_205009', '_SURVEY': '205.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Pozo 1 en Caliza', '_CAD_NUM': '205009', '_CAVENAME': 'P

{'_ID': 3913, '_UID': 3913, '_NAME': 'ENT_211048', '_SURVEY': '211.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Grotte Petite Soeur', '_CAD_NUM': '211048', '_CAVENAME': 'GrottePetiteSoeur', '_LENGTH': 0.0, '_DEPTH': '-4'}
{'_ID': 3914, '_UID': 3914, '_NAME': 'ENT_211049', '_SURVEY': '211.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Puits Octopus', '_CAD_NUM': '211049', '_CAVENAME': 'not known', '_LENGTH': 'not known', '_DEPTH': 'not known'}
{'_ID': 3915, '_UID': 3915, '_NAME': 'ENT_212001', '_SURVEY': '212.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'Grotte de Tarlton', '_CAD_NUM': '212001', '_CAVENAME': 'GrotteDeTarlton', '_LENGTH': nan, '_DEPTH': nan}
{'_ID': 3916, '_UID': 3916, '_NAME': 'ENT_212002', '_SURVEY': '212.UltimaPatagonia', '_SURFACE': 1, '_FIXED': 1, '_ENTRANCE': 0, '_CONTINUA_': 0, '_COMMENT': 'TAR01', '_CAD_NUM': '212002', '_CAVENA

In [175]:
geojson = open("../data/gis/points_fixes.js", "w")
geojson.write("var pointsFixes = \n")
geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
geojson.close()

In [155]:
MAUVAIS_NOM = []

for dirpath, dirnames, filenames in os.walk('../data'):
    # check along each directory.
    for name in dirnames:
        #print(name)
        # find all cave folders, each has a three digit cadastre sector identifier in path
        if not re.match('\d{3}', name):
            if re.match('.*\d{3}',dirpath) and (name !='outputs'):
                print('yay!', name,dirpath)
                try:
                    print(name)
                    #print(NOUVELLE_SYNTHESE[NOUVELLE_SYNTHESE['NomCadastre']==name])
                except KeyError:
                    pass
                    print(name)
                    print("cette grotte n'est pas dans le xls")
                    MAUVAIS_NOM.append(name)

yay! MD-BDG05 ../data/211
MD-BDG05
yay! GouffreAguilucho ../data/211
GouffreAguilucho
yay! PorcheDeLaMadre2 ../data/211
PorcheDeLaMadre2
yay! AbrigoDelViento ../data/211
AbrigoDelViento
yay! MD-BDGS04 ../data/211
MD-BDGS04
yay! MD-BDGS03 ../data/211
MD-BDGS03
yay! GrottePetiteSoeur ../data/211
GrottePetiteSoeur
yay! PuitsOctopus ../data/211
PuitsOctopus
yay! MD-BDGS02 ../data/211
MD-BDGS02
yay! IgueDeDob ../data/211
IgueDeDob
yay! GouffreDeLArche ../data/211
GouffreDeLArche
yay! GrandAvenTriple ../data/211
GrandAvenTriple
yay! GrotteDuNomade ../data/211
GrotteDuNomade
yay! GrotteGrandeSoeur ../data/211
GrotteGrandeSoeur
yay! GrotteMarion ../data/211
GrotteMarion
yay! AvenDeLaTraverse ../data/211
AvenDeLaTraverse
yay! GrotteDeLaConduite ../data/211
GrotteDeLaConduite
yay! GrotteFilostik ../data/211
GrotteFilostik
yay! GouffreDesMousquetaires ../data/211
GouffreDesMousquetaires
yay! Gouffre36 ../data/211
Gouffre36
yay! PorcheDeLImpatience ../data/211
PorcheDeLImpatience
yay! MeandreDesSa

yay! GB01 ../data/201/C4SansNom4
GB01
yay! Cueva10H10 ../data/208
Cueva10H10
yay! Grotte20PlageDeLaBaleine ../data/208
Grotte20PlageDeLaBaleine


In [60]:
IN_FOLDER = []
for NAME in NOUVELLE_SYNTHESE['NomCadastre']:
    
    COND = False
    for dirpath, dirnames, filenames in os.walk('../data'):
    # check along each directory.
        for name in dirnames:
            if NAME in name:
                print(NAME,name,dirpath)
                COND = True
    IN_FOLDER.append((NAME,COND))

LeMasqueEtLaPlume LeMasqueEtLaPlume ../data/213
AbrigoDelViento AbrigoDelViento ../data/211
SumideroPiquante SumideroPiquante ../data/211
GouffreAguilucho GouffreAguilucho ../data/211
ExsurgencePicaflor ExsurgencePicaflor ../data/211
PorcheDeLImpatience PorcheDeLImpatience ../data/211
SimaDelCol SimaDelCol ../data/211
MD-BDGS01 MD-BDGS01 ../data/211
MD-BDGS02 MD-BDGS02 ../data/211
MD-BDGS03 MD-BDGS03 ../data/211
MD-BDGS04 MD-BDGS04 ../data/211
MD-BDG05 MD-BDG05 ../data/211
PleinCintre GrotteDuPleinCintre ../data/209
EmergenceBarillet EmergenceBarillet ../data/209
GouffreDeLaGachette GouffreDeLaGachette ../data/209
ResurgenceDeLaPlage ResurgenceDeLaPlage ../data/209
GrotteEvier GrotteEvier ../data/209
BobLEponge BobLEponge ../data/209
CuevaGringa CuevaGringa ../data/209
Cueva10H10 Cueva10H10 ../data/208
AbriCotier AbriCotier ../data/205
Grotte20PlageDeLaBaleine Grotte20PlageDeLaBaleine ../data/208
AbriDuCerfeuil AbriDuCerfeuil ../data/205
CuevaAyayema CuevaAyayema ../data/206
GrotteDuDy

Cueva6 C4Cueva6 ../data/201
Perte1Lobos Perte1Lobos ../data/211
GrotteAudeline GrotteAudeline ../data/205
PerteDesLobos PerteDesLobos ../data/211
PerteDesLobos PerteDesLobos ../data/ovni
ElCuloDeLaMadre ElCuloDeLaMadre ../data/211
LaGrotte LaGrotte ../data/211
IgueDeDob IgueDeDob ../data/211
GrottePetiteSoeur GrottePetiteSoeur ../data/211
PuitsOctopus PuitsOctopus ../data/211
GouffreDuKit GouffreDuKit ../data/209
GouffreCumpleAnos GouffreCumpleAnos ../data/211
GouffreCumpleAnos GouffreCumpleAnos ../data/209
LaMarmotte LaMarmotte ../data/209
AvenKinka4 AvenKinka4 ../data/209
Perte3 Perte3 ../data/201
AvenKinka3 AvenKinka3 ../data/209
Aven GrandAvenTriple ../data/211
Aven AvenDeLaTraverse ../data/211
Aven ResurgenceDeLAvenir ../data/304
Aven AvenKinka4 ../data/209
Aven AvenKinka3 ../data/209
Aven AvenKinka2 ../data/209
Aven AvenKinka1 ../data/209
Aven GrandAvenPerte ../data/209
Aven GrandAvenTriple ../data/211
Aven AvenDeLaTraverse ../data/211
Aven ResurgenceDeLAvenir ../data/304
Aven Av

EnDesespoirDeCause EnDesespoirDeCause ../data/109
P10 P10 ../data/109
PetiteGrotte PetiteGrotte ../data/109
GouffreHalal GouffreHalal ../data/109
Gouffralail Gouffralail ../data/109
GouffreMaPremiere GouffreMaPremiere ../data/109
GrandeTraversee GrandeTraversee ../data/109
CuevaDeLaMerienda CuevaDeLaMerienda ../data/109
Gauges Gauges ../data/109
Pissotiere Pissotiere ../data/109
TroisEntreesPlusUne TroisEntreesPlusUne ../data/109
QuatreEntreesPlusUne QuatreEntreesPlusUne ../data/109
PuntaBlanca PuntaBlanca ../data/107
TrouDeLaChattiere TrouDeLaChattiere ../data/109
ResurgenceHongo ResurgenceHongo ../data/107
PN01 PN01 ../data/107
PN02 PN02 ../data/107
GouffreJackpot GouffreJackpot ../data/107
PerteCirque4 PerteCirque4 ../data/107
PetiteChinoise PetiteChinoise ../data/109
CuevaCascada CuevaCascada ../data/107
ParaisoDeLasDuchas ParaisoDeLasDuchas ../data/109
MoulinReveBleu MoulinReveBleu ../data/401
CuevaEsperanza CuevaEsperanza ../data/401
Traversee GrandeTraversee ../data/109
Resurgen

In [142]:
A_REVOIR = NOUVELLE_SYNTHESE[pd.DataFrame(IN_FOLDER, columns = ['NAME','COND'])['COND']==False]

In [143]:
A_REVOIR.to_csv('../data/BROUILLON_cadastre/Cavites_Non_Attribuees.csv')

In [144]:
A_REVOIR[A_REVOIR['CAD_NUM']=="NAN"].to_csv('../data/BROUILLON_cadastre/NonAttribue.csv')

In [114]:
with open('../data/106/106.th', 'r') as f:
    data = f.readlines()
    for line in data:
        if '##' in line:
            root = line.split('##')[0]
            suffix = line.split('##')[1]
            print(root+ '\n \tstation\t'+ root.split('fix')[1].split('\t')[0]+'\t"{}"\n'.format(suffix.split('\n')[0].strip(' ')))
    f.close()

    fix ENT_106002	490289	4451286	35.0	
 	station	 ENT_106002	"Cueva No Time"

    fix ENT_106003	490289	4451286	3.0	
 	station	 ENT_106003	"Résurgence NoTime"

    fix ENT_106004	490541	4453870	20.0	
 	station	 ENT_106004	"Cueva Nutria"

    fix ENT_106005	490289	4451286	70.0	
 	station	 ENT_106005	"Porche"

    fix ENT_106006	490604	4450548	0	
 	station	 ENT_106006	"Grotte"

    #fix ENT_106007	490183	4452320	66	
 	station	 ENT_106007	"Haciendas Salinas"

    fix ENT_106008 	490080	4452381	96	
 	station	 ENT_106008 	"Caracara"

    fix ENT_106009  488815 4453011 186 
 	station	 ENT_106009  488815 4453011 186 	"Plateau Egg 1, non descendu à cause de blocs instables"

    fix ENT_106010 488956 4452962 189 
 	station	 ENT_106010 488956 4452962 189 	"Plateau Egg 2, environ 40 m avec un méandre rétrécissant"

    fix ENT_106011 488956 4452962 201 
 	station	 ENT_106011 488956 4452962 201 	"PLateau Egg 3, non descendu, blocs instables"

    fix ENT_106012 489317 4452812 224 
 	station	 ENT